In [1]:
!pip install transformers datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
 

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import pandas as pd

test_data = pd.read_csv('/content/drive/MyDrive/Sentiment_Analysis /processed_test_data.csv')
train_data = pd.read_csv('/content/drive/MyDrive/Sentiment_Analysis /processed_train_data.csv')
validate_data = pd.read_csv('/content/drive/MyDrive/Sentiment_Analysis /processed_validate_data.csv')


In [4]:
train_texts = train_data['cleaned_text'].tolist()  # Replace with actual column name
train_labels = train_data['label'].tolist()  # Replace with actual column name

val_texts = validate_data['cleaned_text'].tolist()  # Replace with actual column name
val_labels = validate_data['label'].tolist()  # Replace with actual column name


In [5]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained('t5-small')

def preprocess_data(texts, labels):
    # Tokenize the texts, ensure each element is a string
    inputs = tokenizer([str(text) for text in texts], max_length=512, padding="max_length", truncation=True, return_tensors="pt")

    # Convert labels to a list of strings if they are not already
    # Assuming labels are integers representing sentiment categories
    # You might need to adjust this based on your actual label format
    # For example, if labels are strings like 'positive', 'negative', etc.,
    # then you don't need this conversion step.
    labels = [str(label) for label in labels]

    # Tokenize the labels
    outputs = tokenizer(labels, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    return inputs.input_ids, outputs.input_ids

train_inputs, train_labels = preprocess_data(train_texts, train_labels)
val_inputs, val_labels = preprocess_data(val_texts, val_labels)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained('t5-small')


/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class T5Dataset(Dataset):
    def __init__(self, input_ids, labels):
        self.input_ids = input_ids
        self.labels = labels

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'labels': self.labels[idx]
        }

train_dataset = T5Dataset(train_inputs, train_labels)
val_dataset = T5Dataset(val_inputs, val_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)


In [ ]:

from tqdm import tqdm

from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)
model.train()
for epoch in range(10):  # Adjust epochs as needed
    loop = tqdm(train_loader, leave=True)
    for batch in loop:
        optimizer.zero_grad()
        input_ids = batch['input_ids']
        labels = batch['labels']
        outputs = model(input_ids=input_ids, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

In [ ]:
# prompt: draw a confusion matrics and also show all result like precision recall f1 score

from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# ... (Your existing code) ...

model.eval()  # Set the model to evaluation mode
predictions = []
true_labels = []

with torch.no_grad():
    for batch in tqdm(val_loader):
        input_ids = batch['input_ids']
        labels = batch['labels']
        outputs = model.generate(input_ids=input_ids)

        # Decode the predictions and true labels
        predicted_labels = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outputs]
        actual_labels = [tokenizer.decode(ids, skip_special_tokens=True) for ids in labels]

        predictions.extend(predicted_labels)
        true_labels.extend(actual_labels)


# Convert string labels to numerical labels if needed for confusion matrix
# (adjust based on your label format)
label_mapping = {'0': 0, '1': 1, '2': 2}  # Example mapping

try:
    y_pred_num = [label_mapping[label] for label in predictions]
    y_true_num = [label_mapping[label] for label in true_labels]

    cm = confusion_matrix(y_true_num, y_pred_num)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.show()

    print(classification_report(y_true_num, y_pred_num))

except KeyError as e:
    print(f"Error: Label {e} not found in label_mapping. Please check your labels and label_mapping.")
    print("Predictions:", predictions[:5])  # Print a few predictions for debugging
    print("True Labels:", true_labels[:5])  # Print a few true labels for debugging

except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
model.eval()
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids']
        labels = batch['labels']

        outputs = model.generate(input_ids)
        print("Generated:", tokenizer.decode(outputs[0], skip_special_tokens=True))
        print("Actual:", tokenizer.decode(labels[0], skip_special_tokens=True))


In [ ]:
model.save_pretrained('./t5_model')
tokenizer.save_pretrained('./t5_model')


In [ ]:
from google.colab import files
!zip -r t5_model.zip ./t5_model
files.download('t5_model.zip')
